In this notebook, we'll start to construct our dataset and explore a few modeling options so that later, we can create an optimized version of our code for production

# Dataset and feature construction

## Step 1: Get the high-level structure of our dataset in place

In [978]:
MinuteFrequencyToSubsample = 15
HoursInTradingDay = 6.5
MinutesInHour = 60
RecordsPerDay = int((MinutesInHour * HoursInTradingDay / MinuteFrequencyToSubsample) + 1)
NameOfColumnForTradingDaysInFuture = "Trading days in future"
LabelName = "returns"
CloseColumnName = "Close"
TradingDaysInWeek = 5
TradingDaysInMonth = 21
TradingDaysInYear = 251

# Features
Month = "Month"
Year = "Year"
Day = "Day"
DayName = "Day name"
Quarter = "Quarter"
MinutesSinceOpen = "Minutes since open"
IsSameDayClose = "Is same day close"
NameOfColumnForTradingDaysInFuture = "Trading days in future"
ReturnSinceLastClose = "Return since last close"
ReturnSinceOpen = "Return since open"
ReturnInLast10Minutes = "Return in last 10 minutes"
ReturnInLast30Minutes = "Return in last 30 minutes"
ReturnInLast1Hour = "Return in last 1 hour"
ReturnInLast2Hours = "Return in last 2 hours"
ReturnInLast3Hours = "Return in last 3 hours"
ReturnInLast4Hours = "Return in last 4 hours"
ReturnInLast5Hours = "Return in last 5 hours"
ReturnInLast6Hours = "Return in last 6 hours"
ReturnInLast2Days = "Return in last 2 days"
ReturnInLast3Days = "Return in last 3 days"
ReturnInLast1Week = "Return in last 1 week"
ReturnInLast2Weeks = "Return in last 2 weeks"
ReturnInLast1Month = "Return in last 1 month"
ReturnInLast3Months = "Return in last 3 months"
ReturnInLast6Months = "Return in last 6 months"
ReturnInLast1Year = "Return in last 1 year"
ReturnInLast1AndAHalfYears = "Return in last 1.5 years"
ReturnInLast2Years = "Return in last 2 years"
ReturnInLast3Years = "Return in last 3 years"


TimeframesInTheFutureInTradingDays = [
    0.5 / HoursInTradingDay, # 3:30 PM
    1.0 / HoursInTradingDay, # 3:00 PM
    2.0 / HoursInTradingDay, # 2:00 PM
    3.0 / HoursInTradingDay, # 1:00 PM
    4.0 / HoursInTradingDay, # 12:00 PM
    5.0 / HoursInTradingDay, # 11:00 AM
    6.0 / HoursInTradingDay, # 10:00 AM
    1.0, # 9:30 AM
    2.0,
    3.0,
    4.0,
    5.0
]

In [979]:
import os
from price_modeling.src.main.code.file_based_historical_data_fetcher import FileBasedHistoricalDataFetcher
file_path = os.path.expanduser('~/Documents/projects/spyndicator-data/SPX_1min.txt')
data_fetcher = FileBasedHistoricalDataFetcher(file_path = file_path)

In [980]:
source_data = data_fetcher.get_historical_data()

In [981]:
evaluation_data_candidates = None
evaluation_data = None

In [982]:
from datetime import datetime
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
one_year_ago = datetime.today() - relativedelta(years = 1)
training_data_candidates = source_data[source_data.index < one_year_ago]
evaluation_data_candidates = source_data[source_data.index >= one_year_ago]

In [983]:
candidates = training_data_candidates[training_data_candidates.index.minute % 15 == 0]

In [984]:
def _get_forward_period_for_percentage_return(trading_days_in_future: float) -> int:
    """
    This method calculates the number of future periods in a dataframe that must be looked at, given a sub-sampled dataframe,
    in order to calculate the percentage return after `trading_days_in_future` in the future with respect to any given record

    :param trading_days_in_future (float): The number of trading days in the future that we need to predict for
    :return (int): The number of time periods in the future to look at given a sub-sampled dataframe
    """

    if trading_days_in_future >= 1.0:
        return int(round(trading_days_in_future * RecordsPerDay, 0))

    minutes_ahead = trading_days_in_future * MinutesInHour * HoursInTradingDay
    return int(round(minutes_ahead / MinuteFrequencyToSubsample, 0))

In [985]:
def _get_time_for_prediction_given_timerange_in_future(trading_days_in_future: float) -> str:
    """
    This method calculates the time at which we are making a prediction given the parameter for trading days in the future

    :param trading_days_in_future (float): The number of trading days in the future that we need to predict for
    :return (str): The string representation of the time for which we are predicting
    """

    if trading_days_in_future < 1.0:
        number_of_minutes_until_close = round(trading_days_in_future * MinutesInHour * HoursInTradingDay, 1)
        timestamp = datetime.now()
        timestamp = timestamp.replace(hour = 16, minute = 00)
        timestamp = timestamp - timedelta(minutes = number_of_minutes_until_close)
        return datetime.strftime(timestamp, "%H:%M")

    return "09:30"

In [986]:
def _create_dataset_with_future_percentage_return(candidates: pd.DataFrame, trading_days_in_future: float) -> pd.DataFrame:
    """
    This method is responsible for taking a source list of candidates and creating a new dataset from them, one which contains the
    percentage return of the stock given the timeframe in the future

    :param candidates (obj:`pd.DataFrame`): A dataframe that contains a list of OHLC data
    :param trading_days_in_future (float): The number of trading days in the future that we need to create labels for
    :return (obj:`pd.DataFrame`): A dataframe that contains a datetime column, the stock price, the days in the future, and the percentage return
    """

    time_at_prediction = _get_time_for_prediction_given_timerange_in_future(trading_days_in_future = trading_days_in_future)
    forward_periods_for_percentage_return = _get_forward_period_for_percentage_return(trading_days_in_future = trading_days_in_future)
    percentage_changes = candidates.pct_change(-forward_periods_for_percentage_return).between_time(time_at_prediction, time_at_prediction, inclusive = "both")
    percentage_changes[NameOfColumnForTradingDaysInFuture] = trading_days_in_future
    percentage_changes.rename(columns = {"Close": "returns"}, inplace = True)
    return percentage_changes[["returns", NameOfColumnForTradingDaysInFuture]]

In [987]:
def _create_dataset_with_future_percentage_return(candidates: pd.DataFrame, trading_days_in_future: float) -> pd.DataFrame:
    """
    This method is responsible for taking a source list of candidates and creating a new dataset from them, one which contains the
    percentage return of the stock given the timeframe in the future

    :param candidates (obj:`pd.DataFrame`): A dataframe that contains a list of OHLC data
    :param trading_days_in_future (float): The number of trading days in the future that we need to create labels for
    :return (obj:`pd.DataFrame`): A dataframe that contains a datetime column, the stock price, the days in the future, and the percentage return
    """

    time_at_prediction = _get_time_for_prediction_given_timerange_in_future(trading_days_in_future = trading_days_in_future)
    forward_periods_for_percentage_return = _get_forward_period_for_percentage_return(trading_days_in_future = trading_days_in_future)
    percentage_changes = candidates.pct_change(-forward_periods_for_percentage_return).between_time(time_at_prediction, time_at_prediction, inclusive = "both")
    percentage_changes[NameOfColumnForTradingDaysInFuture] = trading_days_in_future
    percentage_changes.rename(columns = {"Close": LabelName}, inplace = True)
    percentage_changes = percentage_changes[[LabelName, NameOfColumnForTradingDaysInFuture]]
    percentage_changes.dropna(subset = [LabelName], inplace = True)
    percentage_changes = percentage_changes.join(candidates, how = "left")
    return percentage_changes[["Close", NameOfColumnForTradingDaysInFuture, LabelName]]

In [988]:
import numpy as np

In [989]:
def _construct_dataset_with_labels_from_candidates(candidates: pd.DataFrame) -> pd.DataFrame:
    """
    This method constructs a new dataset that uses the data from the candidates dataframe to act as the basis for creating training
    instances. Multiple new instances, one for each future time period that we would like to predict for, are created based on each
    record within the candidates dataframe.

    :param candidates (obj:`pd.DataFrame`): A dataframe that contains OHLC data
    :return (obj:`pd.DataFrame`): A dataframe in which each record represents a point in time and contains labels for the future return
    of the stock price at a specific point in the future (respective to the time of the instance)
    """

    dataset = None
    for prediction_timeframe in TimeframesInTheFutureInTradingDays:
        dataset_with_labels_for_future = _create_dataset_with_future_percentage_return(candidates = candidates, trading_days_in_future = prediction_timeframe)
        if dataset is None:
            dataset = dataset_with_labels_for_future
        else:
            dataset = pd.concat([dataset, dataset_with_labels_for_future])
    dataset.replace([np.inf, -np.inf], np.nan)
    dataset.dropna(subset = [LabelName])
    return dataset

In [997]:
def _add_features_to_base_instances_and_return_data(base_instances: pd.DataFrame, source_data: pd.DataFrame) -> pd.DataFrame:
    """
    This method adds in all of the features that will be used for prediction into the provided dataset

    :param base_instances (obj:`pd.DataFrame`): A dataframe that contains labels and a column for how far in the future that is being predicted
    :param source_data (obj:`pd.DataFrame`): A dataframe that contains all the original data, useful for matching an instance to its closest closing value
    :return (obj:`pd.DataFrame`): A dataframe that contains all features and labels
    """

    def minutes_since_open(row) -> int:
        start_of_day = row.name # Here, the "name" column is actually the index, wich is the datetime of the record
        start_of_day = start_of_day.replace(hour = 9, minute = 30)
        difference = (row.name - start_of_day).total_seconds() / 60 # 60 seconds per minute
        return difference

    def get_return_since_time(time: str, base_instances: pd.DataFrame, name_of_new_column: str = "Returns") -> pd.DataFrame:
        prices_at_target_time = source_data.between_time(time, time, inclusive = "both")
        prices_at_target_time.sort_index(inplace = True)
        prices_at_target_time = pd.merge_asof(base_instances, prices_at_target_time, left_index = True, right_index = True)
        prices_at_target_time[name_of_new_column] = prices_at_target_time[CloseColumnName + "_x"] / prices_at_target_time[CloseColumnName + "_y"] - 1.0
        returns_since_target_time = prices_at_target_time[[name_of_new_column]]
        return returns_since_target_time

    def get_new_dataset_with_returns_for_specified_time(dataset: pd.DataFrame, new_column_name: str, periods: int, start_time_reset: str, end_time_reset: str) -> pd.DataFrame:
        returns = source_data.pct_change(periods)
        if start_time_reset is not None and end_time_reset is not None:
            index = returns.between_time(start_time_reset, end_time_reset, inclusive = "left").index
            returns.loc[index, CloseColumnName] = 0
        returns[new_column_name] = returns[CloseColumnName]
        returns = returns[[new_column_name]]
        return dataset.join(returns, how = "left")

    base_instances.sort_index(inplace = True)
    dataset = base_instances.sort_index()
    dataset[Month] = dataset.index.month
    dataset[Day] = dataset.index.day
    dataset[DayName] = dataset.index.day_name()
    dataset[Year] = dataset.index.year
    dataset[Quarter] = dataset.index.quarter
    dataset[MinutesSinceOpen] = dataset.apply(minutes_since_open, axis = 1)
    dataset[IsSameDayClose] = dataset[NameOfColumnForTradingDaysInFuture] <= 1.0

    returns_since_last_close = get_return_since_time("16:00", base_instances, ReturnSinceLastClose)
    dataset = dataset.join(returns_since_last_close, how = "left")
    dataset.drop_duplicates(inplace = True)

    returns_since_open = get_return_since_time("09:30", base_instances, ReturnSinceOpen)
    dataset = dataset.join(returns_since_open, how = "left")
    dataset.drop_duplicates(inplace = True)

    dataset = get_new_dataset_with_returns_for_specified_time(dataset = dataset, new_column_name = ReturnInLast10Minutes, periods = 10, start_time_reset = "9:30", end_time_reset = "9:40")
    dataset = get_new_dataset_with_returns_for_specified_time(dataset = dataset, new_column_name = ReturnInLast30Minutes, periods = 30, start_time_reset = "9:30", end_time_reset = "10:00")
    dataset = get_new_dataset_with_returns_for_specified_time(dataset = dataset, new_column_name = ReturnInLast1Hour, periods = MinutesInHour, start_time_reset = "9:30", end_time_reset = "10:30")
    dataset = get_new_dataset_with_returns_for_specified_time(dataset = dataset, new_column_name = ReturnInLast2Hours, periods = 2 * MinutesInHour, start_time_reset = "9:30", end_time_reset = "11:30")
    dataset = get_new_dataset_with_returns_for_specified_time(dataset = dataset, new_column_name = ReturnInLast3Hours, periods = 3 * MinutesInHour, start_time_reset = "9:30", end_time_reset = "12:30")
    dataset = get_new_dataset_with_returns_for_specified_time(dataset = dataset, new_column_name = ReturnInLast4Hours, periods = 4 * MinutesInHour, start_time_reset = "9:30", end_time_reset = "13:30")
    dataset = get_new_dataset_with_returns_for_specified_time(dataset = dataset, new_column_name = ReturnInLast5Hours, periods = 5 * MinutesInHour, start_time_reset = "9:30", end_time_reset = "14:30")
    dataset = get_new_dataset_with_returns_for_specified_time(dataset = dataset, new_column_name = ReturnInLast6Hours, periods = 6 * MinutesInHour, start_time_reset = "9:30", end_time_reset = "15:30")

    dataset = get_new_dataset_with_returns_for_specified_time(dataset = dataset, new_column_name = ReturnInLast2Days, periods = 2 * RecordsPerDay, start_time_reset = None, end_time_reset = None)
    dataset = get_new_dataset_with_returns_for_specified_time(dataset = dataset, new_column_name = ReturnInLast3Days, periods = 3 * RecordsPerDay, start_time_reset = None, end_time_reset = None)
    dataset = get_new_dataset_with_returns_for_specified_time(dataset = dataset, new_column_name = ReturnInLast1Week, periods = TradingDaysInWeek * RecordsPerDay, start_time_reset = None, end_time_reset = None)
    dataset = get_new_dataset_with_returns_for_specified_time(dataset = dataset, new_column_name = ReturnInLast2Weeks, periods = 2 * TradingDaysInWeek * RecordsPerDay, start_time_reset = None, end_time_reset = None)
    dataset = get_new_dataset_with_returns_for_specified_time(dataset = dataset, new_column_name = ReturnInLast1Month, periods = TradingDaysInMonth * RecordsPerDay, start_time_reset = None, end_time_reset = None)
    dataset = get_new_dataset_with_returns_for_specified_time(dataset = dataset, new_column_name = ReturnInLast3Months, periods = 3 * TradingDaysInMonth * RecordsPerDay, start_time_reset = None, end_time_reset = None)
    dataset = get_new_dataset_with_returns_for_specified_time(dataset = dataset, new_column_name = ReturnInLast6Months, periods = 6 * TradingDaysInMonth * RecordsPerDay, start_time_reset = None, end_time_reset = None)
    dataset = get_new_dataset_with_returns_for_specified_time(dataset = dataset, new_column_name = ReturnInLast1Year, periods = TradingDaysInYear * RecordsPerDay, start_time_reset = None, end_time_reset = None)
    dataset = get_new_dataset_with_returns_for_specified_time(dataset = dataset, new_column_name = ReturnInLast1AndAHalfYears, periods = int(TradingDaysInYear * 1.5) * RecordsPerDay, start_time_reset = None, end_time_reset = None)
    dataset = get_new_dataset_with_returns_for_specified_time(dataset = dataset, new_column_name = ReturnInLast2Years, periods = 2 * TradingDaysInYear * RecordsPerDay, start_time_reset = None, end_time_reset = None)
    dataset = get_new_dataset_with_returns_for_specified_time(dataset = dataset, new_column_name = ReturnInLast3Years, periods = 3 * TradingDaysInYear * RecordsPerDay, start_time_reset = None, end_time_reset = None)

    del dataset[CloseColumnName]
    return dataset

In [991]:
training_data_candidates = training_data_candidates[training_data_candidates.index.minute % MinuteFrequencyToSubsample == 0]

In [992]:
training_data_candidates = _construct_dataset_with_labels_from_candidates(candidates = training_data_candidates)

In [998]:
training_data = _add_features_to_base_instances_and_return_data(base_instances = training_data_candidates, source_data = source_data)

In [999]:
training_data

,Trading days in future,returns,Month,Day,Day name,Year,Quarter,Minutes since open,Is same day close,Return since last close,...,Return in last 3 days,Return in last 1 week,Return in last 2 weeks,Return in last 1 month,Return in last 3 months,Return in last 6 months,Return in last 1 year,Return in last 1.5 years,Return in last 2 years,Return in last 3 years
Datetime,,,,,,,,,,,,,,,,,,,,,
2007-04-30 09:30:00,1.000000,0.007243,4,30,Monday,2007,2,0.0,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04-30 09:30:00,2.000000,0.005468,4,30,Monday,2007,2,0.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04-30 09:30:00,3.000000,-0.001429,4,30,Monday,2007,2,0.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04-30 09:30:00,4.000000,-0.006751,4,30,Monday,2007,2,0.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04-30 09:30:00,5.000000,-0.007614,4,30,Monday,2007,2,0.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-04 12:00:00,0.615385,-0.007813,3,4,Friday,2022,1,150.0,True,-0.015696,...,-0.002107,-0.004049,-0.021196,-0.022915,-0.010496,-0.015175,-0.046367,-0.028526,-0.090421,-0.055128
2022-03-04 13:00:00,0.461538,0.000365,3,4,Friday,2022,1,210.0,True,-0.007583,...,0.006843,0.008888,-0.006878,-0.015070,-0.008265,-0.005637,-0.040071,-0.003777,-0.083813,-0.046343
2022-03-04 14:00:00,0.307692,-0.004059,3,4,Friday,2022,1,270.0,True,-0.011972,...,-0.001942,0.002043,-0.004052,-0.010798,-0.014294,-0.006528,-0.044757,-0.024705,-0.081942,-0.051050
